# Trabalho Final de Disciplina
## Análise de Dados de Partidas de LoL para Apoio à Equipe Jogadora
**Grupo**:
    * Clovis Henrique da Silva Chedid
    * Pedro Nascimento
    * Ramiro Ramos

O objetivo do presente trabalho é identificar características e padrões de equipes que jogam League of Legends (LoL) e que possam se tornar vantagem competitiva para uma nova equipe, como acontece em outras modalidades de jogos como o basebol.

O trabalho consiste em analisar dados colhidos de partidas realizadas de League of Legends e aplicar diferentes técnicas estatísticas e probabilisticas, sempre buscando identificar quais aspectos são mais relevantes para se obter êxito em uma partida de LoL.

Inicialmente, buscamos entender os dados obtidos para responder ao conjunto de questões:

* Existe algum aspecto ou estratégia de jogo majoritariamente empregado pelas equipes vencedoras?
* Existe algum aspecto ou estratégia de jogo majoritariamente empregado pelas equipes perdedoras?
* Qual a relação mais comum entre aspectos vencedores e perdedores?
* Quais os aspectos mais comuns em um jogo? E quais aqueles que representam um efeito surpresa?
* Existe alguma relação entre o tempo de jogo e a estratégia adotada?
* Existe alguma relação entre o momento de adoção de cada aspecto e a vitória da partida?

Além dessas questões, também buscamos desenvolver um algoritmo de suporte à equipe jogadora.

Finalmente, é importante destacar que, como qualquer análise estatística de dados, as conclusões obtidas neste trabalho refletem as condições como estas partidas com estas equipes foram jogadas. Assim, este não é um resultado universal para jogos de LoL, se aplica à liga EUW do ano de 2017. Para seu emprego, o conjunto de dados deve ser atualizado e os resultados devem ser refinados para as novas condições.

## Dataset de partidas de LoL

### Dados analisados

Para realizar o trabalho, empregamos o dataset público, disponibilizado site Kaggle, em [League of Legends Dataset](https://www.kaggle.com/datasnaek/league-of-legends).

Trata-se de um conjunto de 50,000 partidas organizadas, realizadas na EUW (European Union West), colhidos em 2017. 

Os dados foram colhidos da API pública da [Riot Games](https://developer.riotgames.com/) e os nomes de times e jogadores foram inferidos a partir de sites de terceiros.

### Estrutura de Dados

O dataset principal conta com as características da partida, disposta em arquivo CSV, com as seguintes colunas:

1. Game ID
1. Creation Time (in Epoch format)
1. Game Duration (in seconds)
1. Season ID
1. Winner (1 = team1, 2 = team2)
1. First Baron (1 = team1, 2 = team2, 0 = none)
1. First dragon (1 = team1, 2 = team2, 0 = none)
1. First tower (1 = team1, 2 = team2, 0 = none)
1. First blood (1 = team1, 2 = team2, 0 = none)
1. First inhibitor (1 = team1, 2 = team2, 0 = none)
1. First Rift Herald (1 = team1, 2 = team2, 0 = none)
1. Champion Team 1
1. Sommoner 1 Team 1
1. Sommoner 2 Team 1
1. Champion Team 2
1. Sommoner 1 Team 2
1. Sommoner 2 Team 2
1. The number of tower kills Team 1
1. The number of inhibitor kills Team 1
1. The number of Baron kills Team 1
1. The number of dragon kills Team 1
1. The number of Rift Herald kills Team 1
1. The number of tower kills Team 2
1. The number of inhibitor kills Team 2
1. The number of Baron kills Team 2
1. The number of dragon kills Team 2
1. The number of Rift Herald kills Team 2
1. Ban 1 Team 1
1. Ban 2 Team 1
1. Ban 3 Team 1
1. Ban 4 Team 1
1. Ban 5 Team 1
1. Ban 1 Team 2
1. Ban 2 Team 2
1. Ban 3 Team 2
1. Ban 4 Team 2
1. Ban 5 Team 2


### Dados Complementares

Além do dataset principal, com o registro das partidas, outros 3 datasets foram utilizados no estudo, com menos importância. Trata-se de arquivos cadastrais em formato JSON, com nomes ou descrições de aspectos do jogo e foram utilizados em pontos onde era necessário descrever um resultado com algum significado para o jogo.

## Referências

Existem vários estudos publicados na internet, em especial no site Kaggle, sobre análises dos dados de LoL. Quatro estudos foram utilizados como referência para análise dos dados:

1. https://www.kaggle.com/jaytegge/league-of-legends-data-analysis
    - Campeões mais escolhidos
    - Campeões mais banidos
    - Tipos de campeões mais escolhidos (Lutador, Support, etc)
    - Summoner Spell mais usados
1. https://www.kaggle.com/jaytegge/league-of-legends-logistic-regression-analysis
    - Prever o ganhador da partida a partir dos objetivos (First Baron, First Blood, etc)
    - Usa 2/3 dos dados para ensinar a máquina e confere os outros 1/3
1. https://www.kaggle.com/gulsahdemiryurek/let-s-predict-league-of-legends-match-score
    - Usa Grid Search Method para desenvolver o modelo
1. https://www.kaggle.com/harryren/everything-you-need-to-know-about-lol
    - Grafico da duração do jogo
    - Entendi quase nada
    - Tipo o B) e C)

## Preparação dos dados

Para este trabalho, empregamos a linguagem Python e os frameworks Pandas, NumPy e SKLearn, que possibilitam o tratamento dos dados e sua análise estatística.

Inicialmente, vamos preparar o ambiente com as bibliotecas e parâmetros necessários para a execução do estudo, bem como carregar o dataset principal.

### Bibliotecas

In [42]:
"""Carrregamento das bibliotecas necessárias para o estudo"""

!pip install -U -q mlxtend
!pip install -U -q nltk
import os
import time

import pandas as pd
import numpy as np
import math 
import matplotlib.pyplot as plt
import statistics as stt
import json

pd.set_option('display.max_rows', 150)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None) 
debug_code = True

### Funções de Apoio

In [43]:
"""
    Funções de suporte, apenas para não poluir o código.
    Algumas funções foram tomadas como referência do kernel 
    https://www.kaggle.com/jaytegge/league-of-legends-data-analysis
"""

def idToName(ID, dic):
    """Função para converter um ID no nome do Champion
    :param int ID:: ID do champion
    :param dict dic:: Atributos de um champion, indexado pelo ID

    :return string:: Nome do champion
    """
    champ = dic['name'][ID]
    return champ

def getTag(name, data):
    """Função para recuperar os tags de um champion
    :param string name:: Nome do champion como no arquivo de atributos
    :param dict data: atributos do champion
    
    :return list:: Lista de tags do champion
    """
    tags = data['tags'][name][0]
    return tags

def numToColor(data):
    """Converte o id do time para as cores Azul(1) e Vermelho(2)
    :param int data: Id do time

    return string [blue, red]:: cor do time
    """
    return 'blue' if data == 0 else 'red'

def trace_log(msg, debug = debug_code):
    """ Função para registro na saída padrão uma mensagem
        :param string msg:: Mensagem a ser escrita
        :param boolean debug:: Registrar ou não. Valor padrão vindo da variável global debug_code
        
        :return None
    """
    if debug:
        print(msg)

### Carregamento dos dados e transformação inicial

Os dados estão no arquivo data/games.csv. Este arquivo possui estrutura colunar, separado por virgulas, com a linha inicial de cabeçalho, onde os nomes das colunas são definidas.

In [17]:
"""Leitura do arquivo de dados"""
cwd = os.getcwd()
trace_log('Pasta inicial: {}'.format(cwd))
data_file = os.path.join(cwd, 'data/games.csv')
trace_log('Arquivo de dados: {}'.format(data_file))
pre_data = pd.read_csv(data_file, sep=',')

Pasta inicial: /Users/coichedid/Documents/Mestrado/Coppe/Estudo/Projeto de Jogos/Projeto_Final
Arquivo de dados: /Users/coichedid/Documents/Mestrado/Coppe/Estudo/Projeto de Jogos/Projeto_Final/data/games.csv


In [46]:
"""Leitura dos arquivo complementare de champions"""
cwd = os.getcwd()
trace_log('Pasta inicial: {}'.format(cwd))
data_file = os.path.join(cwd, 'data/champion_info_2.json')
trace_log('Arquivo de dados: {}'.format(data_file))
jDict = pd.read_json(data_file)
champInfo = pd.read_json((jDict['data']).to_json(), orient='index')
champInfo.set_index(['id'], inplace=True)
champInfo.head()

Pasta inicial: /Users/coichedid/Documents/Mestrado/Coppe/Estudo/Projeto de Jogos/Projeto_Final
Arquivo de dados: /Users/coichedid/Documents/Mestrado/Coppe/Estudo/Projeto de Jogos/Projeto_Final/data/champion_info_2.json


,key,name,tags,title
id,,,,
266,Aatrox,Aatrox,"[Fighter, Tank]",the Darkin Blade
103,Ahri,Ahri,"[Mage, Assassin]",the Nine-Tailed Fox
84,Akali,Akali,[Assassin],the Fist of Shadow
12,Alistar,Alistar,"[Tank, Support]",the Minotaur
32,Amumu,Amumu,"[Tank, Mage]",the Sad Mummy


In [45]:
"""Leitura dos arquivo complementare de champions"""
cwd = os.getcwd()
trace_log('Pasta inicial: {}'.format(cwd))
data_file = os.path.join(cwd, 'data/summoner_spell_info.json')
trace_log('Arquivo de dados: {}'.format(data_file))
spellJson = pd.read_json(data_file)
spellInfo = pd.read_json((spellJson['data']).to_json(),orient='index')
spellInfo.head()

Pasta inicial: /Users/coichedid/Documents/Mestrado/Coppe/Estudo/Projeto de Jogos/Projeto_Final
Arquivo de dados: /Users/coichedid/Documents/Mestrado/Coppe/Estudo/Projeto de Jogos/Projeto_Final/data/summoner_spell_info.json


,description,id,key,name,summonerLevel
1,Removes all disables (excluding suppression and airborne) and summoner spell debuffs affecting your champion and lowers the duration of incoming disables by 65% for 3 seconds.,1,SummonerBoost,Cleanse,6
11,"Deals 390-1000 true damage (depending on champion level) to target epic, large, or medium monster or enemy minion. Restores Health based on your maximum life when used against monsters.",11,SummonerSmite,Smite,10
12,"After channeling for 4.5 seconds, teleports your champion to target allied structure, minion, or ward.",12,SummonerTeleport,Teleport,6
13,Restores 50% of your champion's maximum Mana. Also restores allies for 25% of their maximum Mana.,13,SummonerMana,Clarity,1
14,"Ignites target enemy champion, dealing 70-410 true damage (depending on champion level) over 5 seconds, grants you vision of the target, and reduces healing effects on them for the duration.",14,SummonerDot,Ignite,10


In [29]:
"""Primeira visualização dos dados do arquivo"""
columns = pre_data.columns.values
types = pre_data.dtypes
trace_log('Registros lidos: {}'.format(pre_data.count()[0]))
trace_log('Colunas: {}'.format(columns))
# trace_log('5 Linhas: \n{}'.format(pre_data.head(5)))
trace_log('Tipos de dados: \n{}'.format(types))

Registros lidos: 51490
Colunas: ['gameId' 'creationTime' 'gameDuration' 'seasonId' 'winner' 'firstBlood'
 'firstTower' 'firstInhibitor' 'firstBaron' 'firstDragon'
 'firstRiftHerald' 't1_champ1id' 't1_champ1_sum1' 't1_champ1_sum2'
 't1_champ2id' 't1_champ2_sum1' 't1_champ2_sum2' 't1_champ3id'
 't1_champ3_sum1' 't1_champ3_sum2' 't1_champ4id' 't1_champ4_sum1'
 't1_champ4_sum2' 't1_champ5id' 't1_champ5_sum1' 't1_champ5_sum2'
 't1_towerKills' 't1_inhibitorKills' 't1_baronKills' 't1_dragonKills'
 't1_riftHeraldKills' 't1_ban1' 't1_ban2' 't1_ban3' 't1_ban4' 't1_ban5'
 't2_champ1id' 't2_champ1_sum1' 't2_champ1_sum2' 't2_champ2id'
 't2_champ2_sum1' 't2_champ2_sum2' 't2_champ3id' 't2_champ3_sum1'
 't2_champ3_sum2' 't2_champ4id' 't2_champ4_sum1' 't2_champ4_sum2'
 't2_champ5id' 't2_champ5_sum1' 't2_champ5_sum2' 't2_towerKills'
 't2_inhibitorKills' 't2_baronKills' 't2_dragonKills' 't2_riftHeraldKills'
 't2_ban1' 't2_ban2' 't2_ban3' 't2_ban4' 't2_ban5']
Tipos de dados: 
gameId                int64
c

In [28]:
pre_data.head(5)

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,t1_champ1_sum1,t1_champ1_sum2,t1_champ2id,t1_champ2_sum1,t1_champ2_sum2,t1_champ3id,t1_champ3_sum1,t1_champ3_sum2,t1_champ4id,t1_champ4_sum1,t1_champ4_sum2,t1_champ5id,t1_champ5_sum1,t1_champ5_sum2,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t1_ban1,t1_ban2,t1_ban3,t1_ban4,t1_ban5,t2_champ1id,t2_champ1_sum1,t2_champ1_sum2,t2_champ2id,t2_champ2_sum1,t2_champ2_sum2,t2_champ3id,t2_champ3_sum1,t2_champ3_sum2,t2_champ4id,t2_champ4_sum1,t2_champ4_sum2,t2_champ5id,t2_champ5_sum1,t2_champ5_sum2,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,2,8,12,4,432,3,4,96,4,7,11,11,6,112,4,14,11,1,2,3,0,92,40,69,119,141,104,11,4,498,4,7,122,6,4,238,14,4,412,4,3,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,1,119,7,4,39,12,4,76,4,3,10,4,14,35,4,11,10,4,0,2,1,51,122,17,498,19,54,4,12,25,4,14,120,11,4,157,4,14,92,4,7,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,0,18,4,7,141,11,4,267,3,4,68,4,12,38,12,4,8,1,1,1,0,117,40,29,16,53,69,4,7,412,14,4,126,4,12,24,4,11,22,7,4,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,0,57,4,12,63,4,14,29,4,7,61,4,1,36,11,4,9,2,1,2,0,238,67,516,114,31,90,14,4,19,11,4,412,4,3,92,4,14,22,4,7,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,0,19,4,12,29,11,4,40,4,3,119,4,7,134,7,4,9,2,1,3,0,90,64,412,25,31,37,3,4,59,4,12,141,11,4,38,4,12,51,4,7,3,0,0,1,0,86,11,201,122,18


Como pode ser observado, todas as colunas estão tipificadas como inteiro. O primeiro passo na preparação dos dados é tipificar corretamente as colunas do dataset.

Vamos estabelecer o critério abaixo para isso. As colunas que apresentam valores típicos, como domínios ou IDs serão transformadas em dados categorizados, pois facilitam agregações e filtros, datas, como a coluna creationTime, originalmente foram criadas com formato Epoch mas estão sendo interpretadas como inteiros, seria necessário convertê-la para o formato Datetime. Porém, esta informação não é útil por enquanto, a não ser que fossemos fazer uma análise da evolução de um time ao longo do tempo, ainda assim, poderia ser feito com a coluna no formato inteiro.

| Coluna        | Tipo          |
| ------------- |:-------------:|
| gameId | category |
| creationTime | datetime |
| gameDuration | int64 |
| seasonId | category |
| winner | category |
| firstBlood | category |
| firstTower | category |
| firstInhibitor | category |
| firstBaron | category |
| firstDragon | category |
| firstRiftHerald | category |
| t1_champ1id | category |
| t1_champ1_sum1 | category |
| t1_champ1_sum2 | category |
| t1_champ2id | category |
| t1_champ2_sum1 | category |
| t1_champ2_sum2 | category |
| t1_champ3id | category |
| t1_champ3_sum1 | category |
| t1_champ3_sum2 | category |
| t1_champ4id | category |
| t1_champ4_sum1 | category |
| t1_champ4_sum2 | category |
| t1_champ5id | category |
| t1_champ5_sum1 | category |
| t1_champ5_sum2 | category |
| t1_towerKills | int64 |
| t1_inhibitorKills | int64 |
| t1_baronKills | int64 |
| t1_dragonKills | int64 |
| t1_riftHeraldKills | int64 |
| t1_ban1 | int64 |
| t1_ban2 | int64 |
| t1_ban3 | int64 |
| t1_ban4 | int64 |
| t1_ban5 | int64 |
| t2_champ1id | category |
| t2_champ1_sum1 | category |
| t2_champ1_sum2 | category |
| t2_champ2id | category |
| t2_champ2_sum1 | category |
| t2_champ2_sum2 | category |
| t2_champ3id | category |
| t2_champ3_sum1 | category |
| t2_champ3_sum2 | category |
| t2_champ4id | category |
| t2_champ4_sum1 | category |
| t2_champ4_sum2 | category |
| t2_champ5id | category |
| t2_champ5_sum1 | category |
| t2_champ5_sum2 | category |
| t2_towerKills | int64 |
| t2_inhibitorKills | int64 |
| t2_baronKills | int64 |
| t2_dragonKills | int64 |
| t2_riftHeraldKills | int64 |
| t2_ban1 | int64 |
| t2_ban2 | int64 |
| t2_ban3 | int64 |
| t2_ban4 | int64 |
| t2_ban5 | int64

In [62]:
""" Corrigindo os tipos de dados """
## Colunas de categorias
cat_columns = [
    'gameId',
    'seasonId',
    'winner',
    'firstBlood',
    'firstTower',
    'firstInhibitor',
    'firstBaron',
    'firstDragon',
    'firstRiftHerald',
    't1_champ1id',
    't1_champ1_sum1',
    't1_champ1_sum2',
    't1_champ2id',
    't1_champ2_sum1',
    't1_champ2_sum2',
    't1_champ3id',
    't1_champ3_sum1',
    't1_champ3_sum2',
    't1_champ4id',
    't1_champ4_sum1',
    't1_champ4_sum2',
    't1_champ5id',
    't1_champ5_sum1',
    't1_champ5_sum2',
    't2_champ1id',
    't2_champ1_sum1',
    't2_champ1_sum2',
    't2_champ2id',
    't2_champ2_sum1',
    't2_champ2_sum2',
    't2_champ3id',
    't2_champ3_sum1',
    't2_champ3_sum2',
    't2_champ4id',
    't2_champ4_sum1',
    't2_champ4_sum2',
    't2_champ5id',
    't2_champ5_sum1',
    't2_champ5_sum2'
]

## Colunas de Datetime
dt_column = 'creationTime'

## Convertendo as colunas de categoria
data = pre_data.copy() #Mantem uma cópia dos dados originais
for c in cat_columns:
    data[c] = pre_data[c].astype('category')
    
## Excluindo a coluna de data
data = data.drop(dt_column, axis=1)

In [50]:
## Avaliando os dados finais
d = data.describe()
mean_gameDuration = d['gameDuration']['mean']/60
data.describe()

,gameDuration,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t1_ban1,t1_ban2,t1_ban3,t1_ban4,t1_ban5,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
count,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000,51490.000000
mean,1832.362808,5.699359,1.017537,0.372286,1.387182,0.251466,108.319713,108.786094,108.205904,107.630491,109.027287,5.549466,0.985084,0.414547,1.404370,0.240105,108.216294,107.910216,108.690581,108.626044,108.066576
std,512.017696,3.799808,1.263934,0.583934,1.206818,0.433860,102.247492,102.942617,102.660955,103.000610,102.433377,3.860989,1.256284,0.613768,1.224492,0.427151,102.551787,102.870710,102.592145,103.346952,102.756149
min,190.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,1531.000000,2.000000,0.000000,0.000000,0.000000,0.000000,38.000000,38.000000,37.000000,38.000000,38.000000,2.000000,0.000000,0.000000,0.000000,0.000000,38.000000,37.000000,38.000000,38.000000,38.000000
50%,1833.000000,6.000000,1.000000,0.000000,1.000000,0.000000,90.000000,90.000000,90.000000,89.000000,90.000000,6.000000,0.000000,0.000000,1.000000,0.000000,90.000000,90.000000,90.000000,90.000000,90.000000
75%,2148.000000,9.000000,2.000000,1.000000,2.000000,1.000000,141.000000,141.000000,141.000000,141.000000,141.000000,9.000000,2.000000,1.000000,2.000000,0.000000,141.000000,141.000000,141.000000,141.000000,141.000000
max,4728.000000,11.000000,10.000000,5.000000,6.000000,1.000000,516.000000,516.000000,516.000000,516.000000,516.000000,11.000000,10.000000,4.000000,6.000000,1.000000,516.000000,516.000000,516.000000,516.000000,516.000000


In [35]:
data.describe(include=['category'])

,gameId,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,t1_champ1_sum1,t1_champ1_sum2,t1_champ2id,t1_champ2_sum1,t1_champ2_sum2,t1_champ3id,t1_champ3_sum1,t1_champ3_sum2,t1_champ4id,t1_champ4_sum1,t1_champ4_sum2,t1_champ5id,t1_champ5_sum1,t1_champ5_sum2,t2_champ1id,t2_champ1_sum1,t2_champ1_sum2,t2_champ2id,t2_champ2_sum1,t2_champ2_sum2,t2_champ3id,t2_champ3_sum1,t2_champ3_sum2,t2_champ4id,t2_champ4_sum1,t2_champ4_sum2,t2_champ5id,t2_champ5_sum1,t2_champ5_sum2
count,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490
unique,51053,1,2,3,3,3,3,3,3,138,9,9,138,9,9,138,9,9,138,9,9,138,9,9,138,9,9,138,9,9,138,9,9,138,9,9,138,9,9
top,3331381617,9,1,1,1,1,0,2,0,18,4,4,412,4,4,412,4,4,18,4,4,412,4,4,18,4,4,412,4,4,412,4,4,18,4,4,412,4,4
freq,3,51490,26077,26113,25861,23054,20258,24800,26179,1299,28164,22216,1342,27998,22490,1322,28019,22397,1335,27988,22393,1287,27966,22377,1259,28182,22204,1339,28037,22427,1342,28170,22255,1348,28089,22322,1300,27838,22589


Após as transformações de tipos de dados, foram exibidas algumas estatísticas do dataset. 

As partidas duram em média 30 minutos.

Observando a descrição dos campos categóricos, podemos perceber que existem jogos repetidos no dataset. Vamos analisá-los para ver se será necessário excluí-los.

Neste ponto, é necessário entender alguns aspectos do jogo:

* **Champion**: Personagem escolhido por um jogador.
* **First Blood**: Primeira morte promovida em uma partida, gera recompensas.
* **First Tower ou Turret**: Primeira torre tomada de um adversário.
* **First Inhibitor**: Inhibitor é uma estrutura que bloqueia o treinamento de super minions inimigos. Destruir uma estrutura dessa do adversário melhora a sua capacidade de aumento de força.
* **First Baron**: Baron de Nashor é uma criatura neutra que, morto, pode oferecer força extra para o time que o matou. É uma tática de recuperação em uma partida em desvantagem.
* **First Dragon**: Os dragões são criaturas que podem fortificar temporariamente ou permanentemente o time. 
* **Firts Hift Herald**: Outra criatura neutra que quando morta oferece algumas vantagens para o matador e para seu time.
* **Champion Ban**: Tática utilizada pelo capitão do time para melhor controlar as características do time e também da partida. 
* **Summoner Spells**: Habilidades ou magias específicas de cada *champion* utilizadas ao longo da partida.

Outra observação importante é que o dataset apresenta apenas "Time 1" ou "Time 2" como vencedor, não é possível identificar que time foi o vencedor da partida e quantas partidas o time ganhou.

Uma vez que não é possível identificar os times de uma partida, vamos nos concentrar nas combinações de *champions*. Para isso, será necessário executar mais uma transformação nos dados, associando os IDs dos *champions* aos seus nomes.

Além disso, vamos também associar os ids das *summoner spells* as suas descrições.

Por último, vamos criar uma coluna com as tags de cada *champion*.

In [63]:
""" Transformando os IDs de champions em nomes e os IDs das spells em nomes """
## Colunas com IDs de champions
champion_columns = ['t1_champ1id',
                    't1_champ2id',
                    't1_champ3id',
                    't1_champ4id',
                    't1_champ5id',
                    't2_champ1id',
                    't2_champ2id',
                    't2_champ3id',
                    't2_champ4id',
                    't2_champ5id']
## Colunas com IDs de champions banidos
ban_columns = ['t1_ban1',
               't1_ban2',
               't1_ban3',
               't1_ban4',
               't1_ban5',
               't2_ban1',
               't2_ban2',
               't2_ban3',
               't2_ban4',
               't2_ban5']
## Colunas com IDs de spells utilizadas
summoner_spell_columns = ['t1_champ1_sum1',
                          't1_champ1_sum2',
                          't1_champ2_sum1',
                          't1_champ2_sum2',
                          't1_champ3_sum1',
                          't1_champ3_sum2',
                          't1_champ4_sum1',
                          't1_champ4_sum2',
                          't1_champ5_sum1',
                          't1_champ5_sum2',
                          't2_champ1_sum1',
                          't2_champ1_sum2',
                          't2_champ2_sum1',
                          't2_champ2_sum2',
                          't2_champ3_sum1',
                          't2_champ3_sum2',
                          't2_champ4_sum1',
                          't2_champ4_sum2',
                          't2_champ5_sum1',
                          't2_champ5_sum2']

translate_champion_id_columns = champion_columns + ban_columns

translated_data = data.copy()

for c in translate_champion_id_columns:
    translated_data[c] = translated_data[c].apply(lambda x: idToName(x, champInfo))
for c in summoner_spell_columns:
    translated_data[c] = translated_data[c].apply(lambda x: idToName(x, spellInfo))

In [64]:
translated_data[champion_columns].head(5)

,t1_champ1id,t1_champ2id,t1_champ3id,t1_champ4id,t1_champ5id,t2_champ1id,t2_champ2id,t2_champ3id,t2_champ4id,t2_champ5id
0,Vladimir,Bard,Kog'Maw,Master Yi,Viktor,Graves,Xayah,Darius,Zed,Thresh
1,Draven,Irelia,Nidalee,Kayle,Shaco,Malphite,Morgana,Hecarim,Yasuo,Riven
2,Tristana,Kayn,Nami,Rumble,Kassadin,Cassiopeia,Thresh,Jayce,Jax,Ashe
3,Maokai,Brand,Twitch,Orianna,Dr. Mundo,Malzahar,Warwick,Thresh,Riven,Ashe
4,Warwick,Twitch,Janna,Draven,Syndra,Sona,Jarvan IV,Kayn,Kassadin,Caitlyn


In [65]:
translated_data[ban_columns].head(5)

,t1_ban1,t1_ban2,t1_ban3,t1_ban4,t1_ban5,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,Riven,Janna,Cassiopeia,Draven,Kayn,Fiora,Vayne,Karma,Soraka,Caitlyn
1,Caitlyn,Darius,Teemo,Xayah,Warwick,Master Yi,Vayne,Zed,Caitlyn,Illaoi
2,Lulu,Janna,Twitch,Soraka,Blitzcrank,Yasuo,Zed,Kha'Zix,Maokai,Evelynn
3,Zed,Vayne,Ornn,Fiora,Cho'Gath,Camille,Tristana,Kayn,Janna,Caitlyn
4,Malzahar,Lee Sin,Thresh,Morgana,Cho'Gath,Garen,Master Yi,Braum,Darius,Tristana


In [66]:
translated_data[ban_columns].isna().describe()

,t1_ban1,t1_ban2,t1_ban3,t1_ban4,t1_ban5,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
count,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490
unique,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False
freq,51490,51490,51490,51490,51490,51490,51490,51490,51490,51490


In [67]:
translated_data[summoner_spell_columns].head(5)

,t1_champ1_sum1,t1_champ1_sum2,t1_champ2_sum1,t1_champ2_sum2,t1_champ3_sum1,t1_champ3_sum2,t1_champ4_sum1,t1_champ4_sum2,t1_champ5_sum1,t1_champ5_sum2,t2_champ1_sum1,t2_champ1_sum2,t2_champ2_sum1,t2_champ2_sum2,t2_champ3_sum1,t2_champ3_sum2,t2_champ4_sum1,t2_champ4_sum2,t2_champ5_sum1,t2_champ5_sum2
0,Teleport,Flash,Exhaust,Flash,Flash,Heal,Smite,Ghost,Flash,Ignite,Smite,Flash,Flash,Heal,Ghost,Flash,Ignite,Flash,Flash,Exhaust
1,Heal,Flash,Teleport,Flash,Flash,Exhaust,Flash,Ignite,Flash,Smite,Flash,Teleport,Flash,Ignite,Smite,Flash,Flash,Ignite,Flash,Heal
2,Flash,Heal,Smite,Flash,Exhaust,Flash,Flash,Teleport,Teleport,Flash,Flash,Heal,Ignite,Flash,Flash,Teleport,Flash,Smite,Heal,Flash
3,Flash,Teleport,Flash,Ignite,Flash,Heal,Flash,Cleanse,Smite,Flash,Ignite,Flash,Smite,Flash,Flash,Exhaust,Flash,Ignite,Flash,Heal
4,Flash,Teleport,Smite,Flash,Flash,Exhaust,Flash,Heal,Heal,Flash,Exhaust,Flash,Flash,Teleport,Smite,Flash,Flash,Teleport,Flash,Heal


In [68]:
translated_data.dtypes

gameId                category
gameDuration          int64   
seasonId              category
winner                category
firstBlood            category
firstTower            category
firstInhibitor        category
firstBaron            category
firstDragon           category
firstRiftHerald       category
t1_champ1id           category
t1_champ1_sum1        category
t1_champ1_sum2        category
t1_champ2id           category
t1_champ2_sum1        category
t1_champ2_sum2        category
t1_champ3id           category
t1_champ3_sum1        category
t1_champ3_sum2        category
t1_champ4id           category
t1_champ4_sum1        category
t1_champ4_sum2        category
t1_champ5id           category
t1_champ5_sum1        category
t1_champ5_sum2        category
t1_towerKills         int64   
t1_inhibitorKills     int64   
t1_baronKills         int64   
t1_dragonKills        int64   
t1_riftHeraldKills    int64   
t1_ban1               object  
t1_ban2               object  
t1_ban3 

In [69]:
"""Criando a nova coluna com as tags do champion"""
champInfo.set_index(['name'],inplace=True)
for col in champion_columns:
    translated_data[col + '_tags'] = translated_data[col].apply(lambda x: getTag(x, champInfo))
translated_data.head()

,gameId,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,t1_champ1_sum1,t1_champ1_sum2,t1_champ2id,t1_champ2_sum1,t1_champ2_sum2,t1_champ3id,t1_champ3_sum1,t1_champ3_sum2,t1_champ4id,t1_champ4_sum1,t1_champ4_sum2,t1_champ5id,t1_champ5_sum1,t1_champ5_sum2,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t1_ban1,t1_ban2,t1_ban3,t1_ban4,t1_ban5,t2_champ1id,t2_champ1_sum1,t2_champ1_sum2,t2_champ2id,t2_champ2_sum1,t2_champ2_sum2,t2_champ3id,t2_champ3_sum1,t2_champ3_sum2,t2_champ4id,t2_champ4_sum1,t2_champ4_sum2,t2_champ5id,t2_champ5_sum1,t2_champ5_sum2,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5,t1_champ1id_tags,t1_champ2id_tags,t1_champ3id_tags,t1_champ4id_tags,t1_champ5id_tags,t2_champ1id_tags,t2_champ2id_tags,t2_champ3id_tags,t2_champ4id_tags,t2_champ5id_tags
0,3326086514,1949,9,1,2,1,1,1,1,2,Vladimir,Teleport,Flash,Bard,Exhaust,Flash,Kog'Maw,Flash,Heal,Master Yi,Smite,Ghost,Viktor,Flash,Ignite,11,1,2,3,0,Riven,Janna,Cassiopeia,Draven,Kayn,Graves,Smite,Flash,Xayah,Flash,Heal,Darius,Ghost,Flash,Zed,Ignite,Flash,Thresh,Flash,Exhaust,5,0,0,1,1,Fiora,Vayne,Karma,Soraka,Caitlyn,Mage,Support,Marksman,Assassin,Mage,Marksman,Marksman,Fighter,Assassin,Support
1,3229566029,1851,9,1,1,1,1,0,1,1,Draven,Heal,Flash,Irelia,Teleport,Flash,Nidalee,Flash,Exhaust,Kayle,Flash,Ignite,Shaco,Flash,Smite,10,4,0,2,1,Caitlyn,Darius,Teemo,Xayah,Warwick,Malphite,Flash,Teleport,Morgana,Flash,Ignite,Hecarim,Smite,Flash,Yasuo,Flash,Ignite,Riven,Flash,Heal,2,0,0,0,0,Master Yi,Vayne,Zed,Caitlyn,Illaoi,Marksman,Fighter,Assassin,Fighter,Assassin,Tank,Mage,Fighter,Fighter,Fighter
2,3327363504,1493,9,1,2,1,1,1,2,0,Tristana,Flash,Heal,Kayn,Smite,Flash,Nami,Exhaust,Flash,Rumble,Flash,Teleport,Kassadin,Teleport,Flash,8,1,1,1,0,Lulu,Janna,Twitch,Soraka,Blitzcrank,Cassiopeia,Flash,Heal,Thresh,Ignite,Flash,Jayce,Flash,Teleport,Jax,Flash,Smite,Ashe,Heal,Flash,2,0,0,1,0,Yasuo,Zed,Kha'Zix,Maokai,Evelynn,Marksman,Fighter,Support,Fighter,Assassin,Mage,Support,Fighter,Fighter,Marksman
3,3326856598,1758,9,1,1,1,1,1,1,0,Maokai,Flash,Teleport,Brand,Flash,Ignite,Twitch,Flash,Heal,Orianna,Flash,Cleanse,Dr. Mundo,Smite,Flash,9,2,1,2,0,Zed,Vayne,Ornn,Fiora,Cho'Gath,Malzahar,Ignite,Flash,Warwick,Smite,Flash,Thresh,Flash,Exhaust,Riven,Flash,Ignite,Ashe,Flash,Heal,0,0,0,0,0,Camille,Tristana,Kayn,Janna,Caitlyn,Tank,Mage,Marksman,Mage,Fighter,Mage,Fighter,Support,Fighter,Marksman
4,3330080762,2094,9,1,2,1,1,1,1,0,Warwick,Flash,Teleport,Twitch,Smite,Flash,Janna,Flash,Exhaust,Draven,Flash,Heal,Syndra,Heal,Flash,9,2,1,3,0,Malzahar,Lee Sin,Thresh,Morgana,Cho'Gath,Sona,Exhaust,Flash,Jarvan IV,Flash,Teleport,Kayn,Smite,Flash,Kassadin,Flash,Teleport,Caitlyn,Flash,Heal,3,0,0,1,0,Garen,Master Yi,Braum,Darius,Tristana,Fighter,Marksman,Support,Marksman,Mage,Support,Tank,Fighter,Assassin,Marksman
